In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [8]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 0.20, random_state = 0)


In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from sklearn.tree import DecisionTreeClassifier

In [11]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [12]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'min_weight_fraction_leaf': [0.0, 0.05],
    'max_features': ['sqrt', 'log2'],
    'max_leaf_nodes': [None, 30, 50],
    'random_state': [42],
    'min_impurity_decrease': [0.0, 0.01],
    'class_weight': [None, 'balanced'],
    'ccp_alpha': [0.0, 0.01],
    'monotonic_cst': [None]  # Example; specific use case dependent, often left out of generic grids
}
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 20736 candidates, totalling 103680 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': [0.0, 0.01],
                         'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 10, 20, 30],
                         'max_features': ['sqrt', 'log2'],
                         'max_leaf_nodes': [None, 30, 50],
                         'min_impurity_decrease': [0.0, 0.01],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [2, 10, 20],
                         'min_weight_fraction_leaf': [0.0, 0.05],
                         'monotonic_cst': [None], 'random_state': [42],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [13]:

re=grid.cv_results_
grid_predictions = grid.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

In [15]:
# print classification report
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'ccp_alpha': 0.01, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 30, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': 42, 'splitter': 'random'}: 0.9023809523809525


In [17]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[52  6]
 [ 2 20]]


In [18]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.96      0.90      0.93        58
           1       0.77      0.91      0.83        22

    accuracy                           0.90        80
   macro avg       0.87      0.90      0.88        80
weighted avg       0.91      0.90      0.90        80



In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9181034482758621

In [20]:
# Assuming 'grid' is your GridSearchCV object that has been fitted to the data
best_model = grid.best_estimator_

In [21]:
import pickle
filename="RF_G.sav"
pickle.dump(best_model,open(filename,'wb'))

In [22]:
table=pd.DataFrame.from_dict(re)

In [23]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ccp_alpha,param_class_weight,param_criterion,param_max_depth,param_max_features,param_max_leaf_nodes,...,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018152,0.019125,0.003127,0.006253,0.0,None,gini,None,sqrt,None,...,best,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",0.797547,0.795968,0.843750,0.830822,0.791040,0.811825,0.021295,8783
1,0.015735,0.024388,0.006251,0.007656,0.0,None,gini,None,sqrt,None,...,random,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",0.844699,0.842259,0.830503,0.875897,0.857229,0.850117,0.015434,5263
2,0.006262,0.007669,0.003125,0.006250,0.0,None,gini,None,sqrt,None,...,best,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",0.827459,0.813846,0.890137,0.859920,0.870432,0.852359,0.027979,5039
3,0.000000,0.000000,0.006250,0.007655,0.0,None,gini,None,sqrt,None,...,random,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",0.780782,0.840368,0.724638,0.842259,0.816984,0.801006,0.044153,9185
4,0.003125,0.006250,0.003125,0.006250,0.0,None,gini,None,sqrt,None,...,best,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",0.810937,0.875000,0.842259,0.906923,0.870432,0.861110,0.032408,1763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20731,0.003301,0.004053,0.005583,0.004197,0.01,balanced,log_loss,30,log2,50,...,random,"{'ccp_alpha': 0.01, 'class_weight': 'balanced'...",0.689723,0.717156,0.610658,0.724638,0.647739,0.677983,0.043119,17537
20732,0.006938,0.003545,0.006234,0.001596,0.01,balanced,log_loss,30,log2,50,...,best,"{'ccp_alpha': 0.01, 'class_weight': 'balanced'...",0.737352,0.842259,0.738050,0.691176,0.707233,0.743214,0.052675,15265
20733,0.002910,0.002494,0.003288,0.003712,0.01,balanced,log_loss,30,log2,50,...,random,"{'ccp_alpha': 0.01, 'class_weight': 'balanced'...",0.689723,0.717156,0.610658,0.724638,0.647739,0.677983,0.043119,17537
20734,0.008193,0.005504,0.004533,0.005070,0.01,balanced,log_loss,30,log2,50,...,best,"{'ccp_alpha': 0.01, 'class_weight': 'balanced'...",0.737352,0.842259,0.738050,0.691176,0.707233,0.743214,0.052675,15265
